Importing necessary libraries

In [2]:
from ipynb.fs.full.utilities import (
    create_train_test_split,
    preprocess,
    calculate_pixels,
    calculate_weights,
    train
)

import numpy as np

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceCELoss

import torch
from torch.optim import Adam

File path for the data and model outputs

In [3]:
data_dir = 'D:/Projects/LiverSegmentation/NIfTI'
model_dir = 'D:/Projects/LiverSegmentation/Results'

Create a training and testing split since we do not have separate data

In [4]:
files = create_train_test_split(data_dir, 0.2, 0)

Preprocess the data and create Data Loaders for training and testing data

In [5]:
data = preprocess(files, cache=True)

Loading dataset: 100%|██████████| 74/74 [01:17<00:00,  1.04s/it]


Select GPU for training

In [6]:
device = torch.device("cuda:0")

Define the model and its parameters

In [7]:
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16,32,64,128,256),
    strides=(2,2,2,2),
    num_res_units=2,
    norm=Norm.BATCH
).to(device)

Calculate the ratio of Background : Foreground pixels. Run just once and use the values later on as this takes ~17 mins to run

In [8]:
# count = calculate_pixels(data[0])

Define the Loss Function and the Optimizer for training the model

In [9]:
loss = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(np.array([290, 22])).to(device))
optim = Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

Training the model

In [ ]:
# train(model, data, loss, optim, 600, model_dir)
train(model, data, loss, optim, 600, model_dir, load=True)